In [96]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [174]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
token_index = -1
wrapper()

['W', 0, 'program']
program
['I', 0, 'trivial']
['R', 4, ';']
end of program
['W', 1, 'var']
var
Fatal: expexted identifier 
['I', 1, 'a']
var
['R', 3, ':']
['W', 3, 'integer']
var_type
['R', 4, ';']
['I', 2, 'b']
var
['R', 1, ',']
['I', 3, 'c']
['R', 3, ':']
['W', 7, 'array']
var_type
['R', 7, '[']
['C', 0, '1']
interval
['R', 2, '..']
['C', 1, '10']
['R', 1, ',']
['C', 2, '0']
interval
['R', 2, '..']
['C', 3, '2']
['R', 8, ']']
['W', 8, 'of']
['W', 5, 'string']
['R', 4, ';']
['W', 11, 'begin']
var
Fatal: expexted identifier 


In [103]:
def scan():
    global tokens_chain, token_index, nxtsymb
    token_index += 1
    if token_index < len(tokens_chain):
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [137]:
def antiscan():
    global tokens_chain, token_index, nxtsymb
    token_index -= 1
    if token_index >= 0:
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [130]:
def error(text):
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL)
#     ahah/0

In [116]:
def check(response):
    if len(response):
        error(response)
        return False
    else:
        return True

In [175]:
def wrapper():
    global nxtsymb
    scan()
    if nxtsymb[2] == 'program':
        check(program())
    print('end of program')
    scan()
    if nxtsymb[2] == 'var':
        scan()
        check(var())
        scan()
        if nxtsymb[0] == 'I':
            while check(var()):
                scan()       
    

In [168]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'expected identifier'
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [169]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'expexted identifier'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'expexted identifier'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    check(var_type())
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [170]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return 'expected "["'
        scan()
        check(interval())
        scan()
        while nxtsymb[2] == ',':
            scan()
            check(interval())
            scan()
        if nxtsymb[2] != ']':
            return 'expected "]"'
        scan()
        if nxtsymb[2] != 'of':
            return 'expected "of"'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'expected variable type'
        return ''
    else:
        return 'expected variable type of array description'
        
            

In [171]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'expected constant'
    scan()
    if nxtsymb[2] != '..':
        return 'expected ".."'
    scan()
    if nxtsymb[0] != 'C':
        return 'expected constant'
    return ''

In [161]:
def expression():
    global nxtsymb
    
    term()
    scan()
    while nxtsymb[2] == '+':
        scan()
        factor()
    return ''

In [172]:
def term():
    global nxtsymb
    
    factor()
    while nxtsymb[2] == '*':
        scan()
        factor()
    return ''

In [173]:
def factor():
    global nxtsymb
    
    if nxtsymb[2] == '(':
        scan()
        expression()
        if nxtsymb[2] != ')':
            return 'expected ")"'
        else:
            scan()
    else:
        if nxtsymb[0] not in ['I', 'C']:
            return 'expected constant or identifier'
        scan()
    return ''